# Load dataset

In [12]:
#Importing libraries 
import pandas as pd

## Importing the OS and JSON Modules
import os,json

In [2]:
# Load in superhero info 
info = pd.read_csv("Data/superhero_info - superhero_info.csv", low_memory = False)
info.head()

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}"
2,Abin Sur|DC Comics,Male,Ungaran,good,No Hair,blue,red,"{'Height': '185.0 cm', 'Weight': '90.0 kg'}"
3,Abomination|Marvel Comics,Male,Human / Radiation,bad,No Hair,green,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"
4,Absorbing Man|Marvel Comics,Male,Human,bad,No Hair,blue,Unknown,"{'Height': '193.0 cm', 'Weight': '122.0 kg'}"


In [3]:
# Load in superhero power
power = pd.read_csv("Data/superhero_powers - superhero_powers.csv", low_memory = False)
power.head()

,hero_names,Powers
0,3-D Man,"Agility,Super Strength,Stamina,Super Speed"
1,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
2,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
3,Abin Sur,Lantern Power Ring
4,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."


# Cleaning the info dataset

In [4]:
info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Hero|Publisher  463 non-null    object
 1   Gender          463 non-null    object
 2   Race            463 non-null    object
 3   Alignment       463 non-null    object
 4   Hair color      463 non-null    object
 5   Eye color       463 non-null    object
 6   Skin color      463 non-null    object
 7   Measurements    463 non-null    object
dtypes: object(8)
memory usage: 29.1+ KB


In [5]:
# Exploring existing format with a few examples
info['Hero|Publisher'].head(2)

0            A-Bomb|Marvel Comics
1    Abe Sapien|Dark Horse Comics
Name: Hero|Publisher, dtype: object

Need to split the Hero and Publisher into two columns

In [8]:
## save the 2 new columns into the dataframe
info[['Hero','Publisher']] = info['Hero|Publisher'].str.split('|',expand=True)
info.head(2)

,Hero|Publisher,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,A-Bomb|Marvel Comics,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Abe Sapien|Dark Horse Comics,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [9]:
## drop the original column 
info = info.drop(columns=['Hero|Publisher'])
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics


In [10]:
## examining a single value from the Measurement col
measure = info.loc[0,"Measurements"]
print(type(measure))
measure

<class 'str'>


"{'Height': '203.0 cm', 'Weight': '441.0 kg'}"

In [13]:
## use .str.replace to replace all single quotes
info['Measurements'] = info['Measurements'].str.replace("'",'"')
## Apply the json.loads to the full column
info['Measurements'] = info['Measurements'].apply(json.loads)
info['Measurements'].head()

0    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
1     {'Height': '191.0 cm', 'Weight': '65.0 kg'}
2     {'Height': '185.0 cm', 'Weight': '90.0 kg'}
3    {'Height': '203.0 cm', 'Weight': '441.0 kg'}
4    {'Height': '193.0 cm', 'Weight': '122.0 kg'}
Name: Measurements, dtype: object

In [14]:
hei_wei = info['Measurements'].apply(pd.Series)
hei_wei

,Height,Weight
0,203.0 cm,441.0 kg
1,191.0 cm,65.0 kg
2,185.0 cm,90.0 kg
3,203.0 cm,441.0 kg
4,193.0 cm,122.0 kg
...,...,...
458,183.0 cm,83.0 kg
459,165.0 cm,52.0 kg
460,66.0 cm,17.0 kg
461,170.0 cm,57.0 kg


In [15]:
# concat hei_wei with original dataframe
info = pd.concat((info, hei_wei), axis = 1)
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Measurements,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,"{'Height': '203.0 cm', 'Weight': '441.0 kg'}",A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,"{'Height': '191.0 cm', 'Weight': '65.0 kg'}",Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


In [16]:
## drop the original column 
info = info.drop(columns=['Measurements'])
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg


In [18]:
## Create new columns and add into the dataframe
info[['Height_in_cm','cm']] = info['Height'].str.split(' ',expand=True)
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,Height_in_cm,cm
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg,203.0,cm
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg,191.0,cm


In [19]:
## Create new columns and add into the dataframe
info[['Weight_in_kg','kg']] = info['Weight'].str.split(' ',expand=True)
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height,Weight,Height_in_cm,cm,Weight_in_kg,kg
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0 cm,441.0 kg,203.0,cm,441.0,kg
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0 cm,65.0 kg,191.0,cm,65.0,kg


In [20]:
## drop the original column 
info = info.drop(columns=['Height', 'Weight', 'kg', 'cm'])
info.head(2)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height_in_cm,Weight_in_kg
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0


In [22]:
# Now we can change the height and weight column to a float
info['Height_in_cm'] = info['Height_in_cm'].astype(float)
info['Weight_in_kg'] = info['Weight_in_kg'].astype(float)

In [26]:
# Lets look at our new datatypes
info.dtypes

Gender           object
Race             object
Alignment        object
Hair color       object
Eye color        object
Skin color       object
Hero             object
Publisher        object
Height_in_cm    float64
Weight_in_kg    float64
dtype: object

In [40]:
## example merge with our 2 dataframes
merged = pd.merge(info, power, left_on='Hero', right_on='hero_names', how='left')
merged.head(3)

,Gender,Race,Alignment,Hair color,Eye color,Skin color,Hero,Publisher,Height_in_cm,Weight_in_kg,hero_names,Powers
0,Male,Human,good,No Hair,yellow,Unknown,A-Bomb,Marvel Comics,203.0,441.0,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
1,Male,Icthyo Sapien,good,No Hair,blue,blue,Abe Sapien,Dark Horse Comics,191.0,65.0,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
2,Male,Ungaran,good,No Hair,blue,red,Abin Sur,DC Comics,185.0,90.0,Abin Sur,Lantern Power Ring


# Cleaning the power dataset

In [41]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 463 entries, 0 to 462
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Gender        463 non-null    object 
 1   Race          463 non-null    object 
 2   Alignment     463 non-null    object 
 3   Hair color    463 non-null    object 
 4   Eye color     463 non-null    object 
 5   Skin color    463 non-null    object 
 6   Hero          463 non-null    object 
 7   Publisher     463 non-null    object 
 8   Height_in_cm  463 non-null    float64
 9   Weight_in_kg  463 non-null    float64
 10  hero_names    463 non-null    object 
 11  Powers        463 non-null    object 
dtypes: float64(2), object(10)
memory usage: 47.0+ KB


In [42]:
## exploding the column of lists
exploded = merged.explode('Powers')
exploded[['Hero','Powers']].head(5)

,Hero,Powers
0,A-Bomb,"Accelerated Healing,Durability,Longevity,Super..."
1,Abe Sapien,"Agility,Accelerated Healing,Cold Resistance,Du..."
2,Abin Sur,Lantern Power Ring
3,Abomination,"Accelerated Healing,Intelligence,Super Strengt..."
4,Absorbing Man,"Cold Resistance,Durability,Energy Absorption,S..."


In [39]:
## adding expand=True
power['Powers'].str.split(',',expand=True)

,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,Agility,Super Strength,Stamina,Super Speed,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,Accelerated Healing,Durability,Longevity,Super Strength,Stamina,Camouflage,Self-Sustenance,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,Agility,Accelerated Healing,Cold Resistance,Durability,Underwater breathing,Marksmanship,Weapons Master,Longevity,Intelligence,Super Strength,...,None,None,None,None,None,None,None,None,None,None
3,Lantern Power Ring,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Accelerated Healing,Intelligence,Super Strength,Stamina,Super Speed,Invulnerability,Animation,Super Breath,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
662,Flight,Energy Blasts,Size Changing,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
663,Cold Resistance,Durability,Longevity,Super Strength,Cryokinesis,Immortality,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
664,Agility,Stealth,Danger Sense,Marksmanship,Weapons Master,Longevity,Intelligence,Telepathy,Energy Blasts,Stamina,...,None,None,None,None,None,None,None,None,None,None
665,Cryokinesis,Telepathy,Magic,Fire Control,Probability Manipulation,Water Control,Terrakinesis,Weather Control,None,None,...,None,None,None,None,None,None,None,None,None,None
